# Exploratory Data Analysis (EDA)
This notebook contains the code for visualizing the data and gaining insights from it.

In [ ]:
# Requiered imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import geopandas as gpd
import json
from matplotlib.lines import Line2D
from tqdm import tqdm
from shapely.geometry import Point
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Read and visualize data
df = pd.read_csv("../Data/Cluster_data.csv")
df

In [ ]:
# Check null values
df.isnull().sum()

The NaN values in geoip.continent_code and hostGeoip.continent_code corresponds actually to the North America(NA) continent. This is easily observed by examining the countries with NaN values, they are all from NA. Let us correct the values

In [ ]:
df['_source.geoip.continent_code'] = df['_source.geoip.continent_code'].fillna('NA')
df['_source.hostGeoip.continent_code'] = df['_source.hostGeoip.continent_code'].fillna('NA')

## General EDA

### Attacks by Continent/Country of Origin

In [ ]:
# BAR CHART CONTINENT OF ORIGIN
# Count the number of attacks from each continent
attack_counts = df['_source.geoip.continent_code'].value_counts()
# Remove Antartica for visualization-->just 94 attacks
attack_counts = attack_counts.drop(['AN'])

# Create a color palette (e.g., using a seaborn color palette)
colors = sns.color_palette("rocket",len(attack_counts))

# Create a bar plot
plt.figure(figsize=(12, 6))
ax = attack_counts.plot(kind='bar', color=colors, width=0.7)  # Adjust width as needed

# Adding grid lines
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.title('Cyberattacks by Continent of Origin', fontsize=14)
plt.xlabel('Continent', fontsize=12)
plt.ylabel('Number of Attacks', fontsize=12)

# Millions variable
ax.set_yticklabels(['{:.1f}M'.format(y/ 1e6) for y in ax.get_yticks()])
# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Add a background color
ax.set_facecolor('#f0f0f0')

plt.show()

In [ ]:
# BAR CHART COUNTRY OF ORIGIN
# Count the number of attacks from each country
attack_counts = df['_source.geoip.country_name'].value_counts().head(10)

# Create a color palette (e.g., using a seaborn color palette)
colors = sns.color_palette("rocket",len(attack_counts))

# Create a bar plot
plt.figure(figsize=(12, 6))
ax = attack_counts.plot(kind='bar', color=colors, width=0.7)  # Adjust width as needed

# Adding grid lines
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.title('Cyberattacks by Country of Origin', fontsize=14)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Number of Attacks', fontsize=12)

# Thousands variable
ax.set_yticklabels(['{:.0f}K'.format(y/ 1e3) for y in ax.get_yticks()])

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Add a background color
ax.set_facecolor('#f0f0f0')

plt.show()


Observe that Asia is the continent with the highest number of cyberattacks, possibly due to the weak policies and regulations on the field. Nevertheless, notice also that USA is in the top! 

In [ ]:
# Read the data for the geographical chart
df_map = pd.read_csv('../Data/Map_origin_attacks.csv')

# Function to extract latitude and longitude using our data format
def extract_lat_lon(row):
    try:
        coordinates = json.loads(row)
        lat = coordinates['lat']
        lon = coordinates['lon']
        return lat, lon
    except (ValueError, KeyError):
        return None, None

In [ ]:
# MAP CHART
# Convert Count column to float value
df_map['Count'] = df_map['Count'].str.replace(',', '').astype(float)

# Extract the longitude and latitude
df_map[['lat', 'lon']] = df_map['Geo Centroid'].apply(extract_lat_lon).apply(pd.Series)

# Convert the DataFrame to a GeoDataFrame
geometry = [Point(lon, lat) for lon, lat in zip(df_map['lon'], df_map['lat'])]
gdf = gpd.GeoDataFrame(df_map, geometry=geometry)

# Load the natural earth dataset as the base map
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# Create a figure and axis
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the 2D world map
world.boundary.plot(ax=ax, linewidth=1)

# Define the color and marker size based on 'Count' values
colors = []
sizes = []

for count in gdf['Count']:
    if 1 <= count < 2e6:
        colors.append('yellow')
        sizes.append(10)
    elif 2e6 <= count < 2.5e6:
        colors.append('orange')
        sizes.append(60)
    elif 2.5e6 <= count < 3.5e6:
        colors.append('darkorange')
        sizes.append(100)
    elif 3.5e6 <= count <= 5e6:
        colors.append('red')
        sizes.append(200)
    else:
        colors.append('gray')  
        sizes.append(10)

# Plot the data points with variable marker size and color
gdf.plot(ax=ax, markersize=sizes, color=colors, alpha=0.5, legend=True)
# Create a custom legend
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='yellow', markersize=4, label='1M to 2M'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='orange', markersize=5, label='2M to 2.5M'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='darkorange', markersize=6, label='2.5M to 3.5M'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=7, label='3.5M to 5M'),
]

# Add the legend to the plot
ax.legend(handles=legend_elements, loc='best', title='Attack Count Range')

# Set axis labels and title
plt.title("Origin of attacks Geographic map")

# Show the map
plt.show()


### Attacks by Continent/Country destination

In [ ]:
# BAR CHART CONTINENT OF ORIGIN
# Count the number of attacks from each continent
attack_counts = df['_source.hostGeoip.continent_code'].value_counts()

# Create a color palette (e.g., using a seaborn color palette)
colors = sns.color_palette("rocket",len(attack_counts))

# Create a bar plot
plt.figure(figsize=(12, 6))
ax = attack_counts.plot(kind='bar', color=colors, width=0.7)  # Adjust width as needed

# Adding grid lines
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.title('Cyberattacks by Continent of Origin', fontsize=14)
plt.xlabel('Continent', fontsize=12)
plt.ylabel('Number of Attacks', fontsize=12)

# Millions variable
ax.set_yticklabels(['{:.1f}M'.format(y/ 1e6) for y in ax.get_yticks()])
# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Add a background color
ax.set_facecolor('#f0f0f0')

plt.show()

In [ ]:
# BAR CHART COUNTRY OF ORIGIN
# Count the number of attacks from each country
attack_counts = df['_source.hostGeoip.country_name'].value_counts().head(10)

# Create a color palette (e.g., using a seaborn color palette)
colors = sns.color_palette("rocket",len(attack_counts))

# Create a bar plot
plt.figure(figsize=(12, 6))
ax = attack_counts.plot(kind='bar', color=colors, width=0.7)  # Adjust width as needed

# Adding grid lines
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.title('Cyberattacks by Country of Origin', fontsize=14)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Number of Attacks', fontsize=12)

# Thousands variable
ax.set_yticklabels(['{:.0f}K'.format(y/ 1e3) for y in ax.get_yticks()])

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Add a background color
ax.set_facecolor('#f0f0f0')

plt.show()


While Asia is the continent most affected by cyberattacks, when we examine it by countries, it becomes evident that the USA is the most affected country.

We have seen that Asia is the continent with the highest number of cyberattacks. Are these attacks targeted to the USA?

In [ ]:
# ATTACKS TO USA
df_USA = df[df['_source.hostGeoip.country_name'] =='United States']
df_USA.reset_index(drop = True,inplace=True)

# Count the number of attacks from each country
attack_counts = df_USA['_source.geoip.continent_code'].value_counts()

# Create a color palette (e.g., using a seaborn color palette)
colors = sns.color_palette("rocket",len(attack_counts))

# Create a bar plot
plt.figure(figsize=(12, 6))
ax = attack_counts.plot(kind='bar', color=colors, width=0.7)  # Adjust width as needed

# Adding grid lines
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.title('Cyberattacks by Continent of Origin', fontsize=14)
plt.xlabel('Continent', fontsize=12)
plt.ylabel('Number of Attacks', fontsize=12)

# Thousands variable
ax.set_yticklabels(['{:.0f}K'.format(y/ 1e3) for y in ax.get_yticks()])
# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Add a background color
ax.set_facecolor('#f0f0f0')

plt.show()

20% of the attacks of Asian origin are targeted to USA.

### Protocol and SSH version

In [ ]:
# PIE CHART PROTOCOLS
# Count the occurrences of each unique value in the column
protocol_counts = df['_source.protocol'].value_counts()

# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(protocol_counts, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# Add legend
plt.legend(['SSH', 'Telnet'], title='Protocols',bbox_to_anchor=(1, 1))

plt.title('Protocol Distribution')
plt.show()


In [ ]:
# PIE CHART PROTOCOLS SSH VERSION
value_counts = df['_source.version'].value_counts()

# Select the top 3 values
top_3 = value_counts.head(3)

# Sum the counts of the remaining values and create a new row for 'OTHERS'
others_count = value_counts[3:].sum()
top_3['Others'] = others_count

# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(top_3, autopct='%1.1f%%', startangle=140,textprops={'fontsize': 7})
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# Add legend
plt.legend(top_3.index, title='Protocols', bbox_to_anchor=(1, 1))

plt.title('Protocol version Distribution')
plt.show()

### Attack duration

In [ ]:
# Computing attack duration
# Convert time to datetime to perform operations and compute the attack duration
df['Attack_duration'] = pd.to_datetime(df['_source.endTime']) - pd.to_datetime(df['_source.startTime'])

# Time distribution
# Convert the Attack_duration to Timedelta objects
df['Attack_duration'] = pd.to_timedelta(df['Attack_duration'])

# Define the time interval groups (1-minute intervals)
time_intervals = pd.to_timedelta(pd.Series([f'{i}T' for i in range(0, 16)]))

# Use cut to categorize the timedelta values into the specified time intervals
df['time_interval'] = pd.cut(df['Attack_duration'], time_intervals)

# Group by the time intervals and calculate the count of time values in each group
grouped = df.groupby('time_interval').size()
# Drop the created column
df = df.drop(['time_interval'], axis = 1)
print(grouped)

Let us visualize the data with a histogram.

In [ ]:
# Histogram from 1 to 15 minutes
# Calculate the time interval values in minutes
time_intervals_minutes = [interval.total_seconds() / 60 for interval in time_intervals]

# Exclude the first interval
time_intervals_minutes = time_intervals_minutes[1:]

# Create the histogram
plt.figure(figsize=(10, 6))
plt.hist(df['Attack_duration'].dt.total_seconds() / 60, bins=time_intervals_minutes, edgecolor='k')
plt.title('Attack Duration')
plt.xlabel('Attack Duration (Minutes)')
plt.ylabel('Frequency')
plt.xticks(time_intervals_minutes)  # Setting the x-axis ticks to match the time intervals
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Histogram from 0 to 1 minutes

new_time_intervals = pd.to_timedelta(pd.Series([f'{i}S' for i in range(0, 61, 5)]))

# Use cut to categorize the 'Attack_duration' values into the specified time intervals
df['new_time_interval'] = pd.cut(df['Attack_duration'], new_time_intervals)

# Group by the new time intervals and calculate the count of time values in each group
new_grouped = df.groupby('new_time_interval').size()
# Drop the created column
df = df.drop(['new_time_interval'], axis=1)
# Calculate the time interval values in seconds
new_time_intervals_seconds = [interval.total_seconds() for interval in new_time_intervals]

# Create the new histogram
plt.figure(figsize=(10, 6))
plt.hist(df['Attack_duration'].dt.total_seconds(), bins=new_time_intervals_seconds, edgecolor='k')
plt.title('Attack Duration (First minute)')
plt.xlabel('Attack Duration (Seconds)')
plt.ylabel('Frequency')
plt.xticks(new_time_intervals_seconds)  # Setting the x-axis ticks to match the time intervals
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

We observed a strong tendency to short attacks.

### User/Password combination

In [ ]:
# Function to extract the password
def extract_password(name):
    name_list = eval(name)  # Convert the string to a list
    if len(name_list) >= 2:
        return name_list[1]
    else:
        return ''

In [ ]:
# WORD CLOUD CHART
# Perform the count of the passwords
password_count =  df['_source.loggedin'].apply(extract_password).value_counts()
# Generate the wordcloud chart
wordcloud = WordCloud(width=800, height=400, background_color='white',
                      colormap='viridis', max_words=200, max_font_size=150,
                      min_font_size=10).generate_from_frequencies(password_count)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

The required character count for a password is crucial information. It's reasonable to assume that longer passwords are harder to guess, making it more challenging for unauthorized access to the intended system.

In [ ]:
# LENGTH PASSWORD DISTRIBUTION
# Function to calculate length
def calculate_length(command):
    return len(command)
# Extract password
passwords = df['_source.loggedin'].apply(lambda x: extract_password(x))
# Compute length
df['length_password'] = passwords.apply(lambda x: calculate_length(x))

# Define the intervals
intervals = [(0,4),(4,7),(7, 10), (10, 13), (13, 16), (16, 19), (19, 22), (22, 25), (25, 28), (28, 31),(31,34),(34,37),
             (37,40),(40,45),(45,50),(50,60),(60,1000),(1000,10000)]

# Use pd.cut to categorize the length into the defined intervals
df['length_password'] = pd.cut(df['length_password'], bins=[i for i, j in intervals], labels=[f"{i}-{j}" for i, j in intervals[:-1]])

# Group by the intervals and count the values in each interval
interval_counts = df['length_password'].value_counts().reset_index()
interval_counts.columns = ['Interval', 'Count']

# Sort the intervals in ascending order
interval_counts = interval_counts.sort_values(by='Interval')

# Reset the index
interval_counts = interval_counts.reset_index(drop=True)

# Display the resulting table
print(interval_counts)

Let us see the most used password by continent

In [ ]:
# Add the password column
df['password'] =  df['_source.loggedin'].apply(extract_password)

# Group the DataFrame by destination continent
grouped = df.groupby('_source.hostGeoip.continent_code')

# Define a function to get the top three passwords for each group
def get_top_passwords(group):
    return group['password'].value_counts().nlargest(3)

# Apply the function to each group and combine the results
top_passwords_by_continent = grouped.apply(get_top_passwords)

# Reset the index to have a clean DataFrame
top_passwords_by_continent = top_passwords_by_continent.reset_index()

# Rename the columns for clarity
top_passwords_by_continent.columns = ['_source.hostGeoip.continent_code', 'password', 'count']

# Display the result
top_passwords_by_continent


### Commands fully emulated by the attackant

In [ ]:
# PIE CHART MOST COMMON COMMANDS
value_counts = df['_source.commands'].value_counts()

# Select the top 3 values
top_3 = value_counts.head(3)

# Sum the counts of the remaining values and create a new row for 'OTHERS'
others_count = value_counts[3:].sum()
top_3['Others'] = others_count

# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(top_3, autopct='%1.1f%%', startangle=140,textprops={'fontsize': 7})
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# Truncate legend labels to the first 20 characters
legend_labels = [label[:20] + '...' if len(label) > 20 else label for label in top_3.index]

# Add legend
plt.legend(legend_labels, title='Commands', bbox_to_anchor=(1, 1))
plt.title('Most used commands')
plt.show()

### IP & Port information

In [ ]:
print("Count of unique IP by the attackans:",df['_source.peerIP'].nunique())# Few cyberattacks in relation with the 5M of attacks
print("Count of unique used port by the attackans:",df['_source.peerPort'].nunique())

In [ ]:
print("Count of unique IP of the one who recieved the attack:",df['_source.hostIP'].nunique()) #--> We have 211 sensors
print("Count of unique used port of the one who recieved the attack:",df['_source.hostPort'].nunique())

## EDA for specific country
Let's carry out the same analysis with the attacks that target Spain.

In [ ]:
# Get the data
df_spain = df[df['_source.hostGeoip.country_name'] =='Spain']
df_spain.reset_index(drop = True,inplace=True)

In [ ]:
# BAR CHART CONTINENT OF ORIGIN
# Count the number of attacks from each continent
attack_counts = df_spain['_source.geoip.continent_code'].value_counts()

# Create a color palette (e.g., using a seaborn color palette)
colors = sns.color_palette("rocket",len(attack_counts))

# Create a bar plot
plt.figure(figsize=(12, 6))
ax = attack_counts.plot(kind='bar', color=colors, width=0.7)  # Adjust width as needed

# Adding grid lines
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.title('Cyberattacks by Continent of Origin', fontsize=14)
plt.xlabel('Continent', fontsize=12)
plt.ylabel('Number of Attacks', fontsize=12)

# Millions variable
ax.set_yticklabels(['{:.1f}M'.format(y/ 1e6) for y in ax.get_yticks()])
# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Add a background color
ax.set_facecolor('#f0f0f0')

plt.show()

In [ ]:
# BAR CHART COUNTRY OF ORIGIN
# Count the number of attacks from each country
attack_counts = df_spain['_source.geoip.country_name'].value_counts().head(10)

# Create a color palette (e.g., using a seaborn color palette)
colors = sns.color_palette("rocket",len(attack_counts))

# Create a bar plot
plt.figure(figsize=(12, 6))
ax = attack_counts.plot(kind='bar', color=colors, width=0.7)  # Adjust width as needed

# Adding grid lines
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.title('Cyberattacks by Country of Origin', fontsize=14)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Number of Attacks', fontsize=12)

# Thousands variable
ax.set_yticklabels(['{:.0f}K'.format(y/ 1e3) for y in ax.get_yticks()])

# Rotate x-axis labels for better readability
plt.xticks(rotation=45, ha='right')

# Add a background color
ax.set_facecolor('#f0f0f0')

plt.show()


In [ ]:
# PIE CHART PROTOCOLS
# Count the occurrences of each unique value in the column
protocol_counts = df_spain['_source.protocol'].value_counts()

# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(protocol_counts, autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# Add legend
plt.legend(['SSH', 'Telnet'], title='Protocols',bbox_to_anchor=(1, 1))

plt.title('Protocol Distribution')
plt.show()


In [ ]:
# PIE CHART PROTOCOLS SSH VERSION
value_counts = df_spain['_source.version'].value_counts()

# Select the top 3 values
top_3 = value_counts.head(3)

# Sum the counts of the remaining values and create a new row for 'OTHERS'
others_count = value_counts[3:].sum()
top_3['Others'] = others_count

# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(top_3, autopct='%1.1f%%', startangle=140,textprops={'fontsize': 7})
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# Add legend
plt.legend(top_3.index, title='Protocols', bbox_to_anchor=(1, 1))

plt.title('Protocol version Distribution')
plt.show()

In [ ]:
# Computing attack duration
# Convert time to datetime to perform operations and compute the attack duration
df_spain['Attack_duration'] = pd.to_datetime(df_spain['_source.endTime']) - pd.to_datetime(df_spain['_source.startTime'])

# Time distribution
# Convert the Attack_duration to Timedelta objects
df_spain['Attack_duration'] = pd.to_timedelta(df_spain['Attack_duration'])

# Define the time interval groups (1-minute intervals)
time_intervals = pd.to_timedelta(pd.Series([f'{i}T' for i in range(0, 16)]))

# Use cut to categorize the timedelta values into the specified time intervals
df_spain['time_interval'] = pd.cut(df_spain['Attack_duration'], time_intervals)

# Group by the time intervals and calculate the count of time values in each group
grouped = df_spain.groupby('time_interval').size()
# Drop the created column
df_spain = df_spain.drop(['time_interval'], axis = 1)
print(grouped)

In [ ]:
# Histogram from 1 to 15 minutes
# Calculate the time interval values in minutes
time_intervals_minutes = [interval.total_seconds() / 60 for interval in time_intervals]

# Exclude the first interval
time_intervals_minutes = time_intervals_minutes[1:]

# Create the histogram
plt.figure(figsize=(10, 6))
plt.hist(df_spain['Attack_duration'].dt.total_seconds() / 60, bins=time_intervals_minutes, edgecolor='k')
plt.title('Attack Duration')
plt.xlabel('Attack Duration (Minutes)')
plt.ylabel('Frequency')
plt.xticks(time_intervals_minutes)  # Setting the x-axis ticks to match the time intervals
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Histogram from 0 to 1 minutes

new_time_intervals = pd.to_timedelta(pd.Series([f'{i}S' for i in range(0, 61, 5)]))

# Use cut to categorize the 'Attack_duration' values into the specified time intervals
df_spain['new_time_interval'] = pd.cut(df_spain['Attack_duration'], new_time_intervals)

# Group by the new time intervals and calculate the count of time values in each group
new_grouped = df_spain.groupby('new_time_interval').size()
# Drop the created column
df_spain = df_spain.drop(['new_time_interval'], axis=1)
# Calculate the time interval values in seconds
new_time_intervals_seconds = [interval.total_seconds() for interval in new_time_intervals]

# Create the new histogram
plt.figure(figsize=(10, 6))
plt.hist(df_spain['Attack_duration'].dt.total_seconds(), bins=new_time_intervals_seconds, edgecolor='k')
plt.title('Attack Duration (First minute)')
plt.xlabel('Attack Duration (Seconds)')
plt.ylabel('Frequency')
plt.xticks(new_time_intervals_seconds)  # Setting the x-axis ticks to match the time intervals
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# WORD CLOUD CHART
# Perform the count of the passwords
password_count =  df_spain['_source.loggedin'].apply(extract_password).value_counts()
# Generate the wordcloud chart
wordcloud = WordCloud(width=800, height=400, background_color='white',
                      colormap='viridis', max_words=200, max_font_size=150,
                      min_font_size=10).generate_from_frequencies(password_count)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# LENGTH PASSWORD DISTRIBUTION
# Function to calculate length
def calculate_length(command):
    return len(command)
# Extract password
passwords = df_spain['_source.loggedin'].apply(lambda x: extract_password(x))
# Compute length
df_spain['length_password'] = passwords.apply(lambda x: calculate_length(x))

# Define the intervals
intervals = [(0,4),(4,7),(7, 10), (10, 13), (13, 16), (16, 19), (19, 22), (22, 25), (25, 28), (28, 31),(31,1000),(1000,100000)]

# Use pd.cut to categorize the length into the defined intervals
df_spain['length_password'] = pd.cut(df_spain['length_password'], bins=[i for i, j in intervals], labels=[f"{i}-{j}" for i, j in intervals[:-1]])

# Group by the intervals and count the values in each interval
interval_counts = df_spain['length_password'].value_counts().reset_index()
interval_counts.columns = ['Interval', 'Count']

# Sort the intervals in ascending order
interval_counts = interval_counts.sort_values(by='Interval')

# Reset the index
interval_counts = interval_counts.reset_index(drop=True)

# Display the resulting table
print(interval_counts)

In [ ]:
# PIE CHART MOST COMMON COMMANDS
value_counts = df_spain['_source.commands'].value_counts()

# Select the top 3 values
top_3 = value_counts.head(3)

# Sum the counts of the remaining values and create a new row for 'OTHERS'
others_count = value_counts[3:].sum()
top_3['Others'] = others_count

# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(top_3, autopct='%1.1f%%', startangle=140,textprops={'fontsize': 7})
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# Truncate legend labels to the first 20 characters
legend_labels = [label[:20] + '...' if len(label) > 20 else label for label in top_3.index]

# Add legend
plt.legend(legend_labels, title='Commands', bbox_to_anchor=(1, 1))
plt.title('Most used commands')
plt.show()

In [ ]:
print("Count of unique IP by the attackans:",df_spain['_source.peerIP'].nunique())
print("Count of unique used port by the attackans:",df_spain['_source.peerPort'].nunique())

In [ ]:
print("Count of unique IP of the one who recieved the attack:",df_spain['_source.hostIP'].nunique()) #--> We have 3 sensors in Spain
print("Count of unique used port of the one who recieved the attack:",df_spain['_source.hostPort'].nunique())